## Test Preprocessor

In [ ]:
from src.text_processing import *

In [2]:
from src.general import *

In [3]:
# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [5]:
text = read_text_file('../data/text3.txt')

In [6]:
chunked_text = add_chunk_markers(text)

## Test Planner

In [8]:
from src.planner import *

In [9]:
import uuid

# Generate a unique invocation ID for this workflow run
invocation_id = str(uuid.uuid4())



In [ ]:
plan = await generate_plan(
    invocation_id=invocation_id,
    model="gpt-4o",
    text=chunked_text, 
    fact = 1, 
    inference = 1, 
    table_name = "plan_metadata", 
    database_file = "../database/mcq_metadata.db")

## Test Controller

In [12]:
from src.controller_helper import *

In [13]:
fact =1
inference = 1
main_idea = 1

In [14]:
task_list = create_task_list(chunked_text, plan, fact, inference, main_idea)

In [ ]:
task = task_list[0]
task.get("question_type", "").lower()

### Test MCQ Generators

In [17]:
from src.mcq_generation import *

In [ ]:
questions = await generate_all_mcqs(task_list, invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    concurrency=30,
                                    )


In [ ]:
questions = await generate_all_mcqs_quality_first(task_list, 
                                                  invocation_id, 
                                    model="gpt-4o", 
                                    mcq_metadata_table_name="mcq_metadata", 
                                    evaluation_metadata_table_name="evaluation_metadata",
                                    ranking_metadata_table_name="ranking_metadata",
                                    database_file="../database/mcq_metadata.db",
                                    max_attempt=3,
                                    candidate_num=5,
                                    concurrency=30,
                                    )

### Test the Quality-first MCQ generation

In [ ]:
ranking_metadata = await generate_mcq_quality_first(
    invocation_id=invocation_id,
    model="gpt-4o",
    task=task_list[0],
    mcq_table_name="mcq_metadata",
    ranking_table_name="ranking_metadata",
    database_file='../database/mcq_metadata.db',
    max_attempt = 3,
    attempt= 1,
    candidate_num = 5
)


## Test the Formatter

In [18]:
from src.formatter import *

In [19]:
reformatted_questions = reformat_mcq_metadata(questions)

## Test the Full Workflow

In [6]:
from src.general import *

# get all the files in data folder
files_to_process =  get_files_in_directory("../data/")

In [7]:
text = read_text_file('../data/text3.txt')

In [8]:
from src.workflow import *

In [ ]:
output = await question_generation_workflow(text, 
                                            fact = 1, 
                                            inference = 1, 
                                            main_idea = 1, 
                                            model = "gpt-5", 
                                            quality_first = True, 
                                            candidate_num =5,
                                            max_attempt_for_single_mcq = 3,
                                            plan_metadata_table_name = "plan_metadata",
                                            mcq_metadata_table_name = "mcq_metadata",
                                            evaluation_metadata_table_name = "evaluation_metadata",
                                            ranking_metadata_table_name = "ranking_metadata",
                                            workflow_metadata_table_name = "workflow_metadata", 
                                            database_file = "../database/mcq_metadata.db",
                                            concurrency = 10)


In [2]:
# Export the output from database to a CSV file 
from src.database_handler import export_table_to_csv

In [ ]:
export_table_to_csv(
    table_name="mcq_metadata",
    file_name="20250530_mcq_metadata.csv",
    database_file="../database/mcq_metadata.db",

)
